In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
import time

# Hàm để lấy thông tin chi tiết từ một URL sản phẩm
def get_product_details(product_url):
    # Khởi tạo trình duyệt
    driver = webdriver.Chrome(options=chrome_options)

    # Truy cập URL sản phẩm
    driver.get(product_url)
    
    # Lấy thông tin từ trang sản phẩm
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.find('h3', class_='article__content__title').text.strip()
    product_ld_script = soup.find('script', {'id': 'productld'})
    product_ld_json = json.loads(product_ld_script.contents[0])
    product_description_div = soup.find('div', class_='content-article')
    product_description = product_description_div.find_all(['p', 'h3'])

    # Tạo danh sách chứa nội dung của từng thẻ p và h3
    content_list = []
    for tag in product_description:
        if tag.find('a', class_='preventdefault'):
            break
        content_list.append(tag.get_text(strip=True, separator='\n'))

    # Ghép các nội dung trong danh sách thành một chuỗi
    product_description_text = '\n'.join(content_list)

    # Tạo dictionary chứa thông tin chi tiết
    product_details = {
        'title': title,
        'name': product_ld_json.get('name', ''),
        'description': product_ld_json.get('description', ''),
        'sku': product_ld_json.get('sku', ''),
        'brand': product_ld_json.get('brand', {}).get('name', ''),
        'product_description': product_description_text,
    }

    # Lấy thông tin từ đoạn script JSON khác
    additional_script = soup.find('script', {'type': 'application/ld+json', 'id': 'productld'})
    additional_script_json = json.loads(additional_script.contents[0])

    # Thêm thông tin từ đoạn script JSON khác vào product_details
    product_details.update(additional_script_json)

    # Đóng trình duyệt
    driver.quit()

    return product_details

# Đường dẫn đến WebDriver
webdriver_path = 'D:/Code/Python/f1_crawlling/chromedriver_win32/chromedriver.exe'

# Tạo các tùy chọn cho trình duyệt
chrome_options = Options()
chrome_options.add_argument('--headless')  # Chạy trình duyệt ẩn danh

# Khởi tạo trình duyệt (ở đây sử dụng Chrome)
driver = webdriver.Chrome(options=chrome_options)

# URL của trang chứa danh sách sản phẩm
url = 'https://www.dienmayxanh.com/tivi#c=1942&o=13&pi=12'

# Truy cập trang danh sách sản phẩm
driver.get(url)

# Lấy nội dung HTML sau khi trang đã được tải qua JavaScript
html_content = driver.page_source

# Tiếp tục xử lý HTML giống như bạn đã làm với BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
ul = soup.find('ul', class_='listproduct')
lis = ul.find_all('li')

# Mảng để lưu trữ các kết quả JSON và JSON product_details
results = []
product_details_results = []

for li_id, li in enumerate(lis):
    product_url = li.find('a', class_='item')['href']
    product_details = get_product_details(product_url)
    product_result = {
        'name': product_details['name'],
        'size': product_details.get('size', ''),
        'resolution': product_details.get('resolution', ''),
        'price_old': product_details.get('price_old', ''),
        'discount': product_details.get('discount', ''),
        'price_sale': product_details.get('price_sale', ''),
        'rating': product_details.get('rating', ''),
        'rating_number': product_details.get('rating_number', ''),
        'product_img': product_details.get('product_img', ''),
    }
    results.append(product_result)
    product_details_results.append({'product_details': product_details})

# Convert to JSON and print
json_result = json.dumps(results, ensure_ascii=False, indent=4)
json_product_details_result = json.dumps(product_details_results, ensure_ascii=False, indent=4)

print(json_result)
print(json_product_details_result)

# Đóng trình duyệt
driver.quit()


TypeError: 'NoneType' object is not subscriptable